# 1.Import Dependecies

In [1]:
#GYM STUFF
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 

#STABLEBASELINES STUFF
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

import numpy as np
import random
import os

# 2.Types Of Spaces

In [2]:
Discrete(3)

Discrete(3)

In [3]:
Box(0,1,shape=(3,3)).sample()

array([[0.8147877 , 0.10468549, 0.0419789 ],
       [0.72295207, 0.49551713, 0.08822739],
       [0.3848263 , 0.09212174, 0.18518786]], dtype=float32)

In [4]:
Box(0,255,shape=(3,3), dtype=int).sample()

array([[194, 223, 181],
       [221,  28, 229],
       [192, 110, 132]])

In [5]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

(1, array([96.05396], dtype=float32))

In [6]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([68.66093], dtype=float32))])

In [7]:
MultiBinary(4).sample()

array([1, 1, 1, 0], dtype=int8)

In [8]:
MultiDiscrete([5,2,2]).sample()

array([3, 1, 0], dtype=int64)

# 3.Building An Environment

- Building an agent to  give the besr shower possible
- Random temp
- optimal temp acc to us is 37-39 degrees

In [11]:
class ShowerEnv(Env):
    def __init__(self):
        # Actions we can take -> down, stay, up
        self.action_space = Discrete(3)
        # Temperature array
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start temp
        self.state = 38 + random.randint(-3,3)
        # Set shower length(episode length)
        self.shower_length = 60
    def step(self,action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += action -1 
        # Reduce shower length by 1 second
        self.shower_length -= 1 
        
        # Calculate reward
        if self.state >=37 and self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        # Check if shower is done
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info
    def render(self):
        #could render smtg of ur own choice
        pass
    def reset(self):
        # Reset shower temperature
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        # Reset shower time
        self.shower_length = 60 
        return self.state

# 4.Test Environment

In [12]:
env=ShowerEnv()

C:\Users\kannan\anaconda3\lib\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [15]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:10
Episode:2 Score:-42
Episode:3 Score:-44
Episode:4 Score:-46
Episode:5 Score:10


# 5.Train Model

In [16]:
log_path = os.path.join('Training', 'Logs')

In [17]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [18]:
model.learn(total_timesteps=4000)

Logging to Training\Logs\PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -24.5    |
| time/              |          |
|    fps             | 69       |
|    iterations      | 1        |
|    time_elapsed    | 29       |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -25.5       |
| time/                   |             |
|    fps                  | 130         |
|    iterations           | 2           |
|    time_elapsed         | 31          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009066481 |
|    clip_fraction        | 0.053       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | -7.58e-05   |

# 6.Save Model

In [19]:
model.save('PPO')

In [24]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

(-12.0, 58.787753826796276)